In [ ]:
# Importing the sqlite3 library
import sqlite3

# Connect to SQLite Database
conn = sqlite3.connect('hospital_database.db')
print("Opened database successfully")



Opened database successfully


In [ ]:
# to enable foreign keys usage
conn.execute("PRAGMA foreign_keys = ON;")

# Tables in hospital databases


1. Location
2. Department Location
3. Project Location
4. Employee
5. Department
6. Project
7. Employee Project
8. Dependent
9. Annual Reward
10. Patient
11. Membership
12. Finance








# Creating the tables

Start by creating a table without any foriegn keys.
Location table will include -
1. Location of the departments (e.g Emergency department, radiation department etc.)
2. Location of the hospital branches.
3. Location of the projects(e.g. vaccination projects can be anywhere across the country.)





In [ ]:

conn.execute('''
CREATE TABLE "LOCATION" (
  "location" TEXT PRIMARY KEY
);
''')

* Department location tables connects department number with its location. For example, if a patient want to access an emergecy department but if it is full, she can easily be refered to another branch's EM department by checking this table.
* The primary key of this table is a composite primary key which is formed by the unique combination of department number and the department location.

In [ ]:
conn.execute('''
CREATE TABLE "DEPARTMENT_LOCATION" (
  dlocation TEXT NOT NULL,
  dnumber INTEGER NOT NULL,
  PRIMARY KEY (dnumber, dlocation)
  FOREIGN KEY (dlocation) REFERENCES "LOCATION" ("location")
);
''')

Just like the departments, projects can exist in multiple locations and by summarizing their location in one table, we can easily analyze datas such as which part of the country has better health access and  which diseases are endemic.

In [ ]:
conn.execute('''
CREATE TABLE "PROJECT LOCATION" (
  plocation TEXT NOT NULL,
  pnumber INTEGER NOT NULL,
  PRIMARY KEY (plocation, pnumber),
  FOREIGN KEY (plocation) REFERENCES "LOCATION" ("location")
);
''')

* In employee table, since name can have duplicates, the unique social service number (ssn) is set as a primary key.
* Name is divided into three columns : fname for first name, minit for middle name and lname for lastname.
* While information such as address, salary and department number become mandatory to fill , the rest are set as optional.
* superssn is also included to see the supervisor of each employ, which helps to visualize the hierarchical pattern of the organization.



In [ ]:
# remove the foreign key, d_no, form the table, since it is cycling.
# however, employee's ssn primary key will be referenced as a foreign key from the department table to preserve the connections.
conn.execute('''
CREATE TABLE "EMPLOYEE" (
  ssn INTEGER PRIMARY KEY,
  fname TEXT NOT NULL,
  minit TEXT,
  lname TEXT NOT NULL,
  address TEXT NOT NULL,
  bdate TEXT,
  salary REAL NOT NULL,
  sex CHAR,
  dno INTEGER NOT NULL,
  superssn INTEGER NOT NULL,
  FOREIGN KEY (superssn) REFERENCES "EMPLOYEE" (ssn)
);
''')


In [ ]:
conn.execute('''
CREATE TABLE "DEPARTMENT" (
  dnumber INTEGER PRIMARY KEY,
  dname TEXT NOT NULL,
  dwebsite TEXT NOT NULL,
  funding REAL NOT NULL,
  mgr_ssn INTEGER NOT NULL,
  FOREIGN KEY (mgr_ssn) REFERENCES "EMPLOYEE" (ssn)
);
''')

Project table will describe all the detail information of the project.

In [ ]:
conn.execute('''
CREATE TABLE "PROJECT" (
  pnumber INTEGER PRIMARY KEY,
  pname TEXT NOT NULL,
  plocation TEXT NOT NULL,
  dwebpage TEXT,
  expense REAL NOT NULL,
  dnum INTEGER NOT NULL,
  FOREIGN KEY (plocation) REFERENCES "PROJECT LOCATION" (plocation),
  FOREIGN KEY (dnum) REFERENCES "DEPARTMENT" (dnumber)
);
''')

To reflect performance of the employees, employ project is created. This table will include how many hours do the employ contribute to his project, employee id to specify the employee and project id to specify the project. To connect the project with the individual employees, the unique employee specific project_id is set as a primary key.

In [ ]:
conn.execute('''
CREATE TABLE "EMPLOYEE_PROJECT" (
  project_id INTEGER PRIMARY KEY,
  hours INTEGER NOT NULL,
  ssn INTEGER NOT NULL,
  pnumber INTEGER NOT NULL,
  FOREIGN KEY (ssn) REFERENCES "EMPLOYEE" (ssn),
  FOREIGN KEY (pnumber) REFERENCES "PROJECT" (pnumber)
);
''')

Even though this tables is named dependent, it will act as a guardian table in case of patients and dependent table in case of employees.


In [ ]:
conn.execute('''
CREATE TABLE "DEPENDENT" (
  dependent_name TEXT NOT NULL,
  "dependent_SSN" INTEGER,
  sex TEXT NOT NULL,
  bdate TEXT,
  relationship TEXT,
  essn INTEGER NOT NULL,
  PRIMARY KEY (dependent_name, "dependent_SSN"),
  FOREIGN KEY (essn) REFERENCES "EMPLOYEE" (ssn)
);
''')

Every organization hosts an award system to promote employee satisfication. This table will connect employ names, his ssn and his winning projects.

In [ ]:
conn.execute('''
CREATE TABLE "ANNUAL REWARD" (
  "rewardName" TEXT PRIMARY KEY,
  winner_ssn INTEGER NOT NULL,
  dnumber INTEGER NOT NULL,
  pnumber INTEGER NOT NULL,
  FOREIGN KEY (winner_ssn) REFERENCES "EMPLOYEE" (ssn),
  FOREIGN KEY (dnumber) REFERENCES "DEPARTMENT" (dnumber),
  FOREIGN KEY (pnumber) REFERENCES "PROJECT" (pnumber)
);
''')



*  This table is the most detailed and largest table of the whole data set.
*  It will store all the necessay details such as blood type, address, sex, membership, hospitalization duration, department location as not null datatype.



In [ ]:
conn.execute('''
CREATE TABLE "PATIENT" (
  patient_id INTEGER PRIMARY KEY,
  patient_name TEXT NOT NULL,
  "blood type" TEXT NOT NULL,
  address TEXT NOT NULL,
  sex TEXT NOT NULL,
  membership BOOLEAN NOT NULL,
  "period of stay" TEXT NOT NULL,
  project_number INTEGER NOT NULL,
  dept_category INTEGER NOT NULL,
  ward_location TEXT NOT NULL,
  guardian_name TEXT NOT NULL,
  "EMPLOYEE_ssn" INTEGER NOT NULL,
  guardian_id INTEGER,
  FOREIGN KEY (project_number) REFERENCES "PROJECT" (pnumber),
  FOREIGN KEY (dept_category) REFERENCES "DEPARTMENT" (dnumber),
  FOREIGN KEY (ward_location) REFERENCES "LOCATION" ("location"),
  FOREIGN KEY (guardian_name, guardian_id) REFERENCES "DEPENDENT" (dependent_name, "dependent_SSN"),
  FOREIGN KEY ("EMPLOYEE_ssn") REFERENCES "EMPLOYEE" (ssn)
);

''')


In [ ]:
conn.execute('''
CREATE TABLE "MEMBERSHIP" (
  membership_id INTEGER PRIMARY KEY,
  period TEXT NOT NULL,
  "PATIENT_patient_id" INTEGER NOT NULL,
  FOREIGN KEY ("PATIENT_patient_id") REFERENCES "PATIENT" (patient_id)
);
''')

In [ ]:
conn.execute('''
CREATE TABLE "FINANCE" (
  finance_id NUMERIC PRIMARY KEY,
  "profit" NUMERIC NOT NULL,
  loss NUMERIC NOT NULL,
  "EMPLOYEE_ssn" INTEGER NOT NULL,
  "PROJECT_pnumber" INTEGER NOT NULL,
  "PATIENT_patient_id" INTEGER NOT NULL,
  "MEMBERSHIP_membership_id" INTEGER NOT NULL,
  FOREIGN KEY ("EMPLOYEE_ssn") REFERENCES "EMPLOYEE" (ssn),
  FOREIGN KEY ("PROJECT_pnumber") REFERENCES "PROJECT" (pnumber),
  FOREIGN KEY ("PATIENT_patient_id") REFERENCES "PATIENT" (patient_id),
  FOREIGN KEY ("MEMBERSHIP_membership_id") REFERENCES "MEMBERSHIP" (membership_id)
);
''')

In [ ]:
# Commit the changes and close the connection
conn.commit()
conn.close()

> Sqlite does not support cstring, date, money datatypes. So, cstring and dates are replaced by Text and money is replace by REAL datatypes.

# Checking the created tables

In [ ]:
# check the created tables
# Connect to SQLite Database
conn = sqlite3.connect('hospital_database.db')

# List all tables in the database
cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]
i=1
for table in tables:

  print(f"{i}. {table}")
  i += 1

# Close the database connection
conn.close()

1. LOCATION
2. DEPARTMENT_LOCATION
3. PROJECT LOCATION
4. EMPLOYEE
5. DEPARTMENT
6. PROJECT
7. EMPLOYEE_PROJECT
8. DEPENDENT
9. ANNUAL REWARD
10. PATIENT
11. MEMBERSHIP
12. FINANCE


# Filling the table with datas

In [ ]:
conn = sqlite3.connect('hospital_database.db')  # reopen the database
print("Opened database successfully")

Opened database successfully


In [ ]:
conn.execute("INSERT INTO LOCATION (location) VALUES ('Seoul')");
conn.execute("INSERT INTO LOCATION (location) VALUES ('Incheon')");
conn.execute("INSERT INTO LOCATION (location) VALUES ('Ansan')");
conn.execute("INSERT INTO LOCATION (location) VALUES ('Busan')");
conn.commit()
print("Locations inserted successfully")



Locations inserted successfully


In [ ]:
# print out the rows from location table
cursor = conn.execute("SELECT location from LOCATION")
for row in cursor:
  print("Location", row)




Location ('Seoul',)
Location ('Incheon',)
Location ('Ansan',)
Location ('Busan',)


In [ ]:
# inserting more dummy database into location table
for i in range(4, 30):
    conn.execute(f"INSERT INTO LOCATION (location) VALUES ('Location{i}');")
conn.commit()
print("dummy datas inserted successfully.")


dummy datas inserted successfully.


In [ ]:
# print out new table after adding dummy data
cursor = conn.execute("SELECT location from LOCATION")
for row in cursor:
  print("Location", row)

Location ('Seoul',)
Location ('Incheon',)
Location ('Ansan',)
Location ('Busan',)
Location ('Location4',)
Location ('Location5',)
Location ('Location6',)
Location ('Location7',)
Location ('Location8',)
Location ('Location9',)
Location ('Location10',)
Location ('Location11',)
Location ('Location12',)
Location ('Location13',)
Location ('Location14',)
Location ('Location15',)
Location ('Location16',)
Location ('Location17',)
Location ('Location18',)
Location ('Location19',)
Location ('Location20',)
Location ('Location21',)
Location ('Location22',)
Location ('Location23',)
Location ('Location24',)
Location ('Location25',)
Location ('Location26',)
Location ('Location27',)
Location ('Location28',)
Location ('Location29',)


In [ ]:
# adding 5 rows of dummy data to department location table
for i in range(4,30):
    conn.execute(f"INSERT INTO DEPARTMENT_LOCATION (dlocation,dnumber) VALUES ('Location{i}','{i}');")
conn.commit()
print("dummy datas inserted successfully.")

dummy datas inserted successfully.


In [ ]:
# print out new table after adding dummy data
cursor = conn.execute("SELECT dlocation,dnumber from DEPARTMENT_LOCATION")
for row in cursor:
  print(f"Location :{row[0]}")
  print(f"Dept :{row[1]}\n")


Location :Location4
Dept :4

Location :Location5
Dept :5

Location :Location6
Dept :6

Location :Location7
Dept :7

Location :Location8
Dept :8

Location :Location9
Dept :9

Location :Location10
Dept :10

Location :Location11
Dept :11

Location :Location12
Dept :12

Location :Location13
Dept :13

Location :Location14
Dept :14

Location :Location15
Dept :15

Location :Location16
Dept :16

Location :Location17
Dept :17

Location :Location18
Dept :18

Location :Location19
Dept :19

Location :Location20
Dept :20

Location :Location21
Dept :21

Location :Location22
Dept :22

Location :Location23
Dept :23

Location :Location24
Dept :24

Location :Location25
Dept :25

Location :Location26
Dept :26

Location :Location27
Dept :27

Location :Location28
Dept :28

Location :Location29
Dept :29



In [ ]:
for i in range(4,30):
    conn.execute(f"INSERT INTO 'PROJECT LOCATION' (plocation,pnumber) VALUES ('Location{i}','{i}');")
conn.commit()
print("dummy datas inserted into project location table successfully.")

for i in range(4,30):
    conn.execute(f"INSERT INTO 'EMPLOYEE' (ssn,fname,minit,lname,address,bdate,salary,sex,dno,superssn) VALUES ('{i}','fname{i}','minit{i}','lname{i}','Location{i}','{i+2010}','{i*20000}','F','{i}', '{i}');")
conn.commit()
print("dummy datas inserted into employee table successfully.")





dummy datas inserted into project location table successfully.
dummy datas inserted into employee table successfully.


In [ ]:
# printing out data from employee table
cursor = conn.execute("SELECT ssn, fname, minit,lname,address,bdate,salary,sex,dno,superssn from 'EMPLOYEE'")
for row in cursor:
  print(f"Social Service Number :{row[0]}")
  print(f"First name :{row[1]}")
  print(f"Middle name :{row[2]}")
  print(f"Last name :{row[3]}")
  print(f"Address : {row[4]}")
  print(f"Birth Year :{row[5]}")
  print(f"Salary: {row[6]}")
  print(f"Sex : {row[7]}")
  print(f"Dept number : {row[8]}")
  print(f"Supervisior social service number : {row[9]}")
  print('\n')


Social Service Number :4
First name :fname4
Middle name :minit4
Last name :lname4
Address : Location4
Birth Year :2014
Salary: 80000.0
Sex : F
Dept number : 4
Supervisior social service number : 4


Social Service Number :5
First name :fname5
Middle name :minit5
Last name :lname5
Address : Location5
Birth Year :2015
Salary: 100000.0
Sex : F
Dept number : 5
Supervisior social service number : 5


Social Service Number :6
First name :fname6
Middle name :minit6
Last name :lname6
Address : Location6
Birth Year :2016
Salary: 120000.0
Sex : F
Dept number : 6
Supervisior social service number : 6


Social Service Number :7
First name :fname7
Middle name :minit7
Last name :lname7
Address : Location7
Birth Year :2017
Salary: 140000.0
Sex : F
Dept number : 7
Supervisior social service number : 7


Social Service Number :8
First name :fname8
Middle name :minit8
Last name :lname8
Address : Location8
Birth Year :2018
Salary: 160000.0
Sex : F
Dept number : 8
Supervisior social service number : 8


S

In [ ]:
# conn.execute("DROP TABLE IF EXISTS DEPARTMENT;")

In [ ]:
# Add dummy data into DEPARTMENT
for i in range(4, 30):
    conn.execute(f"INSERT INTO DEPARTMENT (dnumber, dname, dwebsite, funding, mgr_ssn) VALUES ({i}, 'Dept{i}', 'www.{chr(i+65).lower()}dept.com', {i * 10000}, {i});")
conn.commit()
print("Dummy data inserted into DEPARTMENT table successfully.")


Dummy data inserted into DEPARTMENT table successfully.


In [ ]:
# printing out data form DEPARTMENT table
cursor = conn.execute("SELECT dnumber, dname, dwebsite, funding, mgr_ssn from DEPARTMENT")
for row in cursor:
  print(f"Dept no :{row[0]}")
  print(f"Dept name :{row[1]}")
  print(f"Website :{row[2]}")
  print(f"Total funds :{row[3]}")
  print(f"Manager SSN : {row[4]}")
  print('\n')

Dept no :4
Dept name :Dept4
Website :www.edept.com
Total funds :40000.0
Manager SSN : 4


Dept no :5
Dept name :Dept5
Website :www.fdept.com
Total funds :50000.0
Manager SSN : 5


Dept no :6
Dept name :Dept6
Website :www.gdept.com
Total funds :60000.0
Manager SSN : 6


Dept no :7
Dept name :Dept7
Website :www.hdept.com
Total funds :70000.0
Manager SSN : 7


Dept no :8
Dept name :Dept8
Website :www.idept.com
Total funds :80000.0
Manager SSN : 8


Dept no :9
Dept name :Dept9
Website :www.jdept.com
Total funds :90000.0
Manager SSN : 9


Dept no :10
Dept name :Dept10
Website :www.kdept.com
Total funds :100000.0
Manager SSN : 10


Dept no :11
Dept name :Dept11
Website :www.ldept.com
Total funds :110000.0
Manager SSN : 11


Dept no :12
Dept name :Dept12
Website :www.mdept.com
Total funds :120000.0
Manager SSN : 12


Dept no :13
Dept name :Dept13
Website :www.ndept.com
Total funds :130000.0
Manager SSN : 13


Dept no :14
Dept name :Dept14
Website :www.odept.com
Total funds :140000.0
Manager S

In [ ]:
# add dummy data into PROJECT

for i in range(4, 30):
    conn.execute(f"INSERT INTO PROJECT (pnumber, pname, plocation, dwebpage, expense, dnum) VALUES (?,?,?,?,?,?)",(i,f"Project{i}",f"Location{i}",f'www.{chr(i+65).lower()}dept.com',i*100000,i))
conn.commit()
print("Dummy data inserted into PROJECT table successfully.")


Dummy data inserted into PROJECT table successfully.


In [ ]:
# print out the datas in project
cursor = conn.execute("SELECT pnumber, pname, plocation, dwebpage, expense, dnum from PROJECT")
for row in cursor:
  print(f"Project no :{row[0]}")
  print(f"Project name :{row[1]}")
  print(f"Location :{row[2]}")
  print(f"Department webpage :{row[3]}")
  print(f"expense : {row[4]}")
  print(f"Dept Number : {row[5]}")
  print('\n')


Project no :4
Project name :Project4
Location :Location4
Department webpage :www.edept.com
expense : 400000.0
Dept Number : 4


Project no :5
Project name :Project5
Location :Location5
Department webpage :www.fdept.com
expense : 500000.0
Dept Number : 5


Project no :6
Project name :Project6
Location :Location6
Department webpage :www.gdept.com
expense : 600000.0
Dept Number : 6


Project no :7
Project name :Project7
Location :Location7
Department webpage :www.hdept.com
expense : 700000.0
Dept Number : 7


Project no :8
Project name :Project8
Location :Location8
Department webpage :www.idept.com
expense : 800000.0
Dept Number : 8


Project no :9
Project name :Project9
Location :Location9
Department webpage :www.jdept.com
expense : 900000.0
Dept Number : 9


Project no :10
Project name :Project10
Location :Location10
Department webpage :www.kdept.com
expense : 1000000.0
Dept Number : 10


Project no :11
Project name :Project11
Location :Location11
Department webpage :www.ldept.com
expen

In [ ]:
# add dummy data into EMPLOYEE_PROJECT
for i in range(4, 30):
    conn.execute(f"INSERT INTO EMPLOYEE_PROJECT (project_id, hours, ssn, pnumber) VALUES (?,?,?,?)"
    ,(i,i*0.1,i,i))
conn.commit()
print("Dummy data inserted into PROJECT table successfully.")

Dummy data inserted into PROJECT table successfully.


In [ ]:
# print out the project
cursor = conn.execute("SELECT project_id, hours, ssn, pnumber from EMPLOYEE_PROJECT")
for row in cursor:
  print(f"Project no :{row[0]}")
  print(f"Total hours of contribution:{row[1]}")
  print(f"Employee ID :{row[2]}")
  print(f"Project Number :{row[3]}")

  print('\n')

Project no :4
Total hours of contribution:0.4
Employee ID :4
Project Number :4


Project no :5
Total hours of contribution:0.5
Employee ID :5
Project Number :5


Project no :6
Total hours of contribution:0.6000000000000001
Employee ID :6
Project Number :6


Project no :7
Total hours of contribution:0.7000000000000001
Employee ID :7
Project Number :7


Project no :8
Total hours of contribution:0.8
Employee ID :8
Project Number :8


Project no :9
Total hours of contribution:0.9
Employee ID :9
Project Number :9


Project no :10
Total hours of contribution:1
Employee ID :10
Project Number :10


Project no :11
Total hours of contribution:1.1
Employee ID :11
Project Number :11


Project no :12
Total hours of contribution:1.2000000000000002
Employee ID :12
Project Number :12


Project no :13
Total hours of contribution:1.3
Employee ID :13
Project Number :13


Project no :14
Total hours of contribution:1.4000000000000001
Employee ID :14
Project Number :14


Project no :15
Total hours of contri

In [ ]:
# Close the database connection
conn.close()